# Run PESTPP-OPT

In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import flopy
import pyemu


flopy is installed in /Users/jeremyw/Dev/gw1876/activities_2day_mfm/notebooks/flopy


In [2]:
t_d = "template"
m_d = "master_opt"

In [3]:
pst = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))
pst.write_par_summary_table(filename="none").sort_index()

,type,transform,count,initial value,upper bound,lower bound,standard deviation
drncond_k00,drncond_k00,log,10,0,1,-1,0.5
flow,flow,log,1,0,0.09691,-0.124939,0.0554622
grhk3,grhk3,log,705,0,1,-1,0.5
grhk4,grhk4,log,705,0,1,-1,0.5
grhk5,grhk5,log,705,0,1,-1,0.5
grrech2,grrech2,log,705,0,0.0413927,-0.0457575,0.0217875
grrech3,grrech3,log,705,0,0.0413927,-0.0457575,0.0217875
grss3,grss3,log,705,0,1,-1,0.5
grss4,grss4,log,705,0,1,-1,0.5
grss5,grss5,log,705,0,1,-1,0.5


In [4]:
pst.pestpp_options = {}
#dvg = ["welflux_k02","welflux"]
dvg = ["welflux_k02"]
pst.pestpp_options["opt_dec_var_groups"] = dvg
pst.pestpp_options["opt_direction"] = "max"

In [5]:
par = pst.parameter_data
par.loc[:,"partrans"] = "fixed"

#turn off pumping in the scenario
par.loc["welflux_001","parlbnd"] = 0.0 
par.loc["welflux_001","parval1"] = 0.0 
dvg_pars = par.loc[par.pargp.apply(lambda x: x in dvg),"parnme"]
par.loc[dvg_pars,"partrans"] = "none"
par.loc[dvg_pars,"parlbnd"] = 0.0
par.loc[dvg_pars,"parubnd"] = 2.0
par.loc[dvg_pars,"parval1"] = 1.0

pst.rectify_pgroups()
pst.parameter_groups.loc[dvg,"inctyp"] = "absolute"
pst.parameter_groups.loc[dvg,"inctyp"] = "absolute"
pst.parameter_groups.loc[dvg,"derinc"] = 0.25

pst.parameter_groups.loc[dvg,:]

,pargpnme,inctyp,derinc,derinclb,forcen,derincmul,dermthd,splitthresh,splitreldiff,splitaction,extra
pargpnme,,,,,,,,,,,
welflux_k02,welflux_k02,absolute,0.25,0.0,switch,2.0,parabolic,0.00001,0.5,smaller,NaN


### constraints

In [6]:
obs = pst.observation_data
obs.loc[:,"weight"] = 0.0
swgw_hist = obs.loc[obs.obsnme.apply(lambda x: "fa" in x and( "hw" in x or "tw" in x)),"obsnme"]
obs.loc[swgw_hist,:]

,obsnme,obsval,weight,obgnme,extra
obsnme,,,,,
fa_hw_19791230,fa_hw_19791230,-1289.170400,0.0,flaqx,NaN
fa_hw_19801229,fa_hw_19801229,-690.416200,0.0,flaqx,NaN
fa_tw_19791230,fa_tw_19791230,-573.743640,0.0,flaqx,NaN
fa_tw_19801229,fa_tw_19801229,-160.726405,0.0,flaqx,NaN


We need to change the obs group (`obgnme`) so that `pestpp-opt` will recognize these two model outputs as constraints.  lets also assume that the sw-gw flux needs to be at least -1000

In [7]:
obs.loc[swgw_hist,"obgnme"] = "less_than"
obs.loc[swgw_hist,"weight"] = 1.0

obs.loc[swgw_hist,"obsval"] = -300

tot_abs_rate = ["flx_wells_19791230"]#,"flx_wells_19801229"]
obs.loc[tot_abs_rate,"obgnme"] = "less_than"
obs.loc[tot_abs_rate,"weight"] = 1.0
obs.loc[tot_abs_rate,"obsval"] = -600.0
pst.less_than_obs_constraints

obsnme
fa_hw_19791230            fa_hw_19791230
fa_hw_19801229            fa_hw_19801229
fa_tw_19791230            fa_tw_19791230
fa_tw_19801229            fa_tw_19801229
flx_wells_19791230    flx_wells_19791230
Name: obsnme, dtype: object

In [8]:
pst.control_data.noptmax = 1
pst.write(os.path.join(t_d,"freyberg_opt.pst"))

In [9]:
pyemu.os_utils.start_slaves(t_d,"pestpp-opt","freyberg_opt.pst",num_slaves=10,master_dir=m_d)

In [10]:
jco = pyemu.Jco.from_binary(os.path.join(m_d,"freyberg_opt.1.jcb")).to_dataframe().loc[pst.less_than_obs_constraints,:]
jco

,wf0200090016,wf0200110013,wf0200200014,wf0200260010,wf0200290006,wf0200340012
fa_hw_19791230,137.57200,126.32400,46.30000,21.90800,18.12000,4.8320
fa_hw_19801229,22.58400,28.65600,12.03600,12.29200,13.12800,3.3560
fa_tw_19791230,6.50728,14.53516,93.28136,92.42320,71.84608,82.9612
fa_tw_19801229,4.10836,7.60104,15.29948,30.88604,34.79872,17.5232
flx_wells_19791230,-150.00000,-150.00000,-150.00000,-150.00000,-150.00000,-150.0000


In [11]:
par_df = pyemu.pst_utils.read_parfile(os.path.join(m_d,"freyberg_opt.1.par"))
print(par_df.loc[dvg_pars,"parval1"].sum())
par_df.loc[dvg_pars,:]

8.1332977617072


,parnme,parval1,scale,offset
parnme,,,,
wf0200090016,wf0200090016,2.000000,1.0,0.0
wf0200110013,wf0200110013,2.000000,1.0,0.0
wf0200200014,wf0200200014,2.000000,1.0,0.0
wf0200260010,wf0200260010,0.133298,1.0,0.0
wf0200290006,wf0200290006,0.000000,1.0,0.0
wf0200340012,wf0200340012,2.000000,1.0,0.0


In [12]:
pst = pyemu.Pst(os.path.join(m_d,"freyberg_opt.pst"),resfile=os.path.join(m_d,"freyberg_opt.1.rei"))
pst.res.loc[pst.nnz_obs_names,:]

,name,group,measured,modelled,residual,weight
name,,,,,,
fa_hw_19791230,fa_hw_19791230,less_than,-300.0,-699.3735,399.3735,1.0
fa_hw_19801229,fa_hw_19801229,less_than,-300.0,-714.4580,414.4580,1.0
fa_tw_19791230,fa_tw_19791230,less_than,-300.0,-407.7249,107.7249,1.0
fa_tw_19801229,fa_tw_19801229,less_than,-300.0,-299.7868,-0.2132,1.0
flx_wells_19791230,flx_wells_19791230,less_than,-600.0,-1219.9948,619.9948,1.0


In [13]:
#todo chance constraints (fosm and en-based), well pars and constraints in scen period, no-run hotstart option

### Opt under uncertainty part 1: FOSM chance constraints

In [14]:
pst.pestpp_options["opt_risk"] = 0.4

In [15]:
cn_pars = par.loc[par.pargp.apply(lambda x: "cn" in x),"parnme"]
cn_pars

parnme
hk6_cn        hk6_cn
hk7_cn        hk7_cn
hk8_cn        hk8_cn
rech4_cn    rech4_cn
rech5_cn    rech5_cn
ss6_cn        ss6_cn
ss7_cn        ss7_cn
ss8_cn        ss8_cn
strt6_cn    strt6_cn
strt7_cn    strt7_cn
strt8_cn    strt8_cn
sy6_cn        sy6_cn
sy7_cn        sy7_cn
sy8_cn        sy8_cn
vka6_cn      vka6_cn
vka7_cn      vka7_cn
vka8_cn      vka8_cn
Name: parnme, dtype: object

In [16]:
par = pst.parameter_data
par.loc[cn_pars,"partrans"] = "log"
pst.control_data.noptmax = 1
pst.write(os.path.join(t_d,"freyberg_opt_uu1.pst"))
pst.npar_adj

23

In [17]:
pyemu.os_utils.start_slaves(t_d,"pestpp-opt","freyberg_opt_uu1.pst",num_slaves=20,master_dir=m_d)

In [18]:
pst = pyemu.Pst(os.path.join(m_d,"freyberg_opt_uu1.pst"),resfile=os.path.join(m_d,"freyberg_opt_uu1.1.rei"))
pst.res.loc[pst.nnz_obs_names,:]

,name,group,measured,modelled,residual,weight
name,,,,,,
fa_hw_19791230,fa_hw_19791230,less_than,-300.0,-666.13442,366.13442,1.0
fa_hw_19801229,fa_hw_19801229,less_than,-300.0,-682.60800,382.60800,1.0
fa_tw_19791230,fa_tw_19791230,less_than,-300.0,-223.47050,-76.52950,1.0
fa_tw_19801229,fa_tw_19801229,less_than,-300.0,-208.37540,-91.62460,1.0
flx_wells_19791230,flx_wells_19791230,less_than,-600.0,-1586.33800,986.33800,1.0


In [19]:
par_df = pyemu.pst_utils.read_parfile(os.path.join(m_d,"freyberg_opt_uu1.1.par"))
print(par_df.loc[dvg_pars,"parval1"].sum())
par_df.loc[dvg_pars,:]

10.575587155980312


,parnme,parval1,scale,offset
parnme,,,,
wf0200090016,wf0200090016,2.000000,1.0,0.0
wf0200110013,wf0200110013,2.000000,1.0,0.0
wf0200200014,wf0200200014,1.481006,1.0,0.0
wf0200260010,wf0200260010,1.094581,1.0,0.0
wf0200290006,wf0200290006,2.000000,1.0,0.0
wf0200340012,wf0200340012,2.000000,1.0,0.0


### Opt under uncertainty part 2: ensemble-based chance constraints

In [20]:
obs_df = pd.read_csv(os.path.join("master_prior_sweep","sweep_out.csv"),index_col=0)
obs_df = obs_df.loc[obs_df.failed_flag==0,:]

In [21]:
std = obs_df.std().loc[pst.nnz_obs_names]
std

fa_hw_19791230        352.846322
fa_hw_19801229        460.423786
fa_tw_19791230        469.183099
fa_tw_19801229        548.767448
flx_wells_19791230    705.753519
dtype: float64

In [22]:
pst.observation_data.loc[pst.nnz_obs_names,"weight"] = std.loc[pst.nnz_obs_names]
pst.pestpp_options["opt_std_weights"] = True
pst.write(os.path.join(t_d,"freyberg_opt_uu2.pst"))

In [23]:
pyemu.os_utils.start_slaves(t_d,"pestpp-opt","freyberg_opt_uu2.pst",num_slaves=10,master_dir=m_d)

In [24]:
par_df = pyemu.pst_utils.read_parfile(os.path.join(m_d,"freyberg_opt_uu2.1.par"))
print(par_df.loc[dvg_pars,"parval1"].sum())
par_df.loc[dvg_pars,:]

11.038907743747297


,parnme,parval1,scale,offset
parnme,,,,
wf0200090016,wf0200090016,2.000000,1.0,0.0
wf0200110013,wf0200110013,2.000000,1.0,0.0
wf0200200014,wf0200200014,1.038908,1.0,0.0
wf0200260010,wf0200260010,2.000000,1.0,0.0
wf0200290006,wf0200290006,2.000000,1.0,0.0
wf0200340012,wf0200340012,2.000000,1.0,0.0
